In [1]:
import pandas as pd
import numpy as np

import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import sys

import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, validation_curve
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


from IPython.display import Audio

from utils.audio_preprocess import *
from utils.septr import SeparableTr
from utils.feature_exctraction_v2 import get3d_data

In [2]:
from tqdm import tqdm

In [3]:
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D,MaxPooling2D, Flatten, Dropout, BatchNormalization, Input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

#import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
# # ave maria 🙏🏻🙏🏿
from scikeras.wrappers import KerasClassifier

In [5]:
data_path = data_path()

In [6]:
train, test = train_test_split(data_path, random_state=42, shuffle=True)

In [ ]:
x_train, y_train, x_test, y_test = get3d_data(train, test, feats = False)

In [ ]:
# scaling our data with sklearn's Standard scaler
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

### CNN

In [ ]:
def cnn_model(nfilters1=128, nfilters2=64, nfilters3=32, size_kernel=5, nstrides=1, s_pool1=5, s_pool2=5, s_pool3=5):
    model=Sequential()
    model.add(Conv2D(filters=nfilters1, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=s_pool1, strides = nstrides, padding = 'same'))

    model.add(Conv2D(filters=nfilters2, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.15))
    model.add(BatchNormalization())    
    model.add(MaxPooling2D(pool_size=s_pool2, strides = nstrides, padding = 'same'))

    model.add(Conv2D(filters=nfilters3, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=s_pool3, strides = nstrides, padding = 'same'))

    #model.add(Conv1D(filters=nfilters4, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    #model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))

    model.add(Dense(units=6, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.30, verbose=0, patience=2, min_lr=0.0000001)

    #model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test, y_test), callbacks=[rlrp])    
    
    #model.summary()

    return model

In [ ]:
model = KerasClassifier(
    model=cnn_model, 
    verbose=0, 
    batch_size=32, 
    loss="categorical_crossentropy", 
    nfilters1 = 64, 
    nfilters2 = 64, 
    nfilters3 = 64, 
    nfilters4 = 64, 
    size_kernel = 3,
    nstrides = 1,
    s_pool1 = 5,
    s_pool2 = 5,
    s_pool3 = 5
    )

In [ ]:
param_grid = {
    'nfilters1': [256, 128, 64, 32],
    'nfilters2': [256, 128, 64, 32],
    'nfilters3': [256, 128, 64, 32],
    'nfilters4': [256, 128, 64, 32],
    'size_kernel': [3, 5],
    'nstrides': [1, 2, 3],
    's_pool1': [3, 5, 8],
    's_pool2': [3, 5, 8],
    's_pool3': [3, 5, 8]
    #'optimizer': ['adam']
}

# Grid
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=3, verbose=3, n_jobs=3, n_iter=1000) #GridSearchCV
grid_result = grid.fit(x_train, y_train)

# Print
print("Best hyperparameter: ", grid_result.best_params_)
print("Best accuracy: ", grid_result.best_score_)

best_model = grid_result.best_estimator_
val_accuracy = best_model.score(x_test, y_test)
print("Validaton accuracy: ", val_accuracy)

In [ ]:
cnn_model().fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

Epoch 1/20
469/666 [====================>.........] - ETA: 2:17:16 - loss: 6.1889 - accuracy: 0.1703

### CapNet

In [ ]:
from tensorflow.keras import layers, models

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, capsule_dim, routings=3, kernel_initializer='glorot_uniform', **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.capsule_dim = capsule_dim
        self.routings = routings
        self.kernel_initializer = kernel_initializer

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(
            shape=(input_dim, self.num_capsules, self.capsule_dim),
            initializer=self.kernel_initializer,
            trainable=True,
        )

    def call(self, inputs):
        # Inputs shape: (batch_size, input_dim, input_num_capsules)
        # W shape: (input_dim, num_capsules, capsule_dim)
        # u_hat: (batch_size, input_dim, num_capsules, capsule_dim)
        u_hat = tf.matmul(inputs, self.W)
        
        # Initialize b with zero values
        b = tf.zeros((inputs.shape[0], input_dim, self.num_capsules))
        
        # Routing algorithm
        for _ in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            s = tf.reduce_sum(c * u_hat, axis=1)
            v = squash(s)
            b += tf.reduce_sum(u_hat * v[:, tf.newaxis, :, :], axis=-1)

        return v
    
def squash(vector):
    vector_norm = tf.norm(vector, axis=-1, keepdims=True)
    vector_squashed = (vector_norm / (1 + vector_norm**2)) * vector
    return vector_squashed

def cn_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(x_train.shape[1:])))  # Input shape
    model.add(CapsuleLayer(num_capsules=10, capsule_dim=16))  # Primary Capsules
    model.add(CapsuleLayer(num_capsules=10, capsule_dim=16))  # Digit Capsules
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


In [ ]:
cn_model().fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test))


ValueError: Exception encountered when calling layer "capsule_layer" (type CapsuleLayer).

in user code:

    File "/var/folders/45/bzqc3zcj52bbs_vq26k_nt8c0000gn/T/ipykernel_13066/962544458.py", line 23, in call  *
        u_hat = tf.matmul(inputs, self.W)

    ValueError: Dimensions must be equal, but are 1 and 10 for '{{node capsule_layer/MatMul}} = BatchMatMulV2[T=DT_FLOAT, adj_x=false, adj_y=false](Placeholder, capsule_layer/MatMul/ReadVariableOp)' with input shapes: [?,128,219,1], [1,10,16].


Call arguments received by layer "capsule_layer" (type CapsuleLayer):
  • inputs=tf.Tensor(shape=(None, 128, 219, 1), dtype=float32)

### SepTr